In [9]:
import pandas as pd

from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

In [11]:
gigachad = GigaChat(credentials='595554e2-2f71-40d5-a98a-88fa47b127d2', scope="GIGACHAT_API_PERS")